In [3]:
from google.colab import drive
drive.mount('/content/drive/')

rootDir = '/content/drive/MyDrive/'

TIMESTAMP = '2022_03_30'

dataPath = rootDir + 'TP2/Datasets/Recipe1M/' + TIMESTAMP
tarPath = rootDir + 'Colab Notebooks/recime/data/' + TIMESTAMP
weightDir = rootDir + 'Colab Notebooks/recime/weights/'
logDir = rootDir + 'Colab Notebooks/recime/runs/titleTrainer/'


import sys
sys.path.append(rootDir + 'Colab Notebooks/recime/LSTM')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
import sys
sys.path.append('D:/02_Studium/SBX/mad-recime/network/LSTM')

tarPath = 'D:/02_Studium/SBX/mad-recime/data/2022_03_19'

In [5]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.seed import seed_everything

import random
import re
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.pytorch_lightning import (
    TuneReportCallback,
    TuneReportCheckpointCallback,
)



In [6]:
import src.nlp_utils.model as models


In [7]:
import src.nlp_utils.data_module as data_module

import importlib

In [8]:
# set seed to get consistent results, deactivate if random results are wanted
seed_everything(42)

Global seed set to 42


42

In [9]:
importlib.reload(models)
importlib.reload(data_module)

from config import create_config

config = create_config({})

titleMod = data_module.TitleDataModule(tarPath)
titleMod.setup()

config['vocabSize'] = titleMod.vocab_size

model = models.EmbedLSTM(config)

trainer = Trainer(max_epochs=config['epochs'])
trainer.fit(model, titleMod)

D:\02_Studium/SBX/mad-recime/network/LSTM\src\preProc.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))
D:\02_Studium/SBX/mad-recime/network/LSTM\src\preProc.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseFrame = baseFrame.append(pd.read_pickle(dataSetSplits[split]))
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
C:\Users\Hannes\miniconda3\envs\madEnv\lib\site-packages\pytorch_lightning\core\datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name      | Type             | Params
--------------------------

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

# Test

In [ ]:
# mnist = MNISTDataModule(my_path)
# model = LitClassifier()

# trainer = Trainer()
# trainer.fit(model, mnist)

# Optimization

In [ ]:
# %%

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.05,
    patience=3,
    verbose=False,
    mode="min",
    divergence_threshold=3.00,
)

cwd = os.getcwd()
save_folder = os.path.join(cwd, "../logs/StancePrediction_SemEval")


class MyPrintingCallback(Callback):
    def on_fit_start(self, trainer, pl_module):
        print("Starting to train!")

    def on_fit_end(self, trainer, pl_module):
        print("Finished training")

    def on_test_start(self, trainer, pl_module):
        print("Start to test")

    def on_test_end(self, trainer, pl_module):
        print("Finished testing")


checkpoint_callback = ModelCheckpoint(
    monitor="val_epoch_stance_F1",
    filename="{epoch}-{val_loss:.2f}-{val_epoch_stance_F1:.2f}",
    save_top_k=3,
    mode="max",
)

callback = TuneReportCallback(
    {"loss": "val_loss", "mean_F1": "val_epoch_F1"}, on="validation_end"
)


# training loop that tests out different hyperparameters and saves the results into the log folder
def train_tune(config, callbacks, epochs=10, gpus=0):
    data_module = TitleDateModule(num_workers=4, config=config)
    data_module.setup("")
    config["vocabSize"] = data_module.vocab_size
    model = CustomDistilBertModel(config)
    trainer = pl.Trainer(
        gpus=gpus,
        log_every_n_steps=1,
        flush_logs_every_n_steps=1,
        callbacks=callbacks,
        deterministic=True,
        default_root_dir=save_folder,
        max_epochs=epochs,
    )  # gradient_clip_val=0.5, stochastic_weight_avg=True, check_val_every_n_epoch=10, num_sanity_val_steps=2, overfit_batches=0.01
    # logger=TensorBoardLogger(save_dir=tune.get_trial_dir(), name="", version="."),
    trainer.fit(model, datamodule=data_module)
    # might not be called due to scheduler and reporter which cancel training early if results don't look promising
    trainer.test(model, datamodule=data_module)


# config with radom sampling for learning rate and batch size
config = {
    "dataset_path": "../../data/raw/SemEval/",
    "learning_rate": tune.sample_from(lambda: abs(random.gauss(1e-3, 1e-3))),
    "batch_size": tune.choice([16, 32, 64, 128]),
    "epochs": 20,
    "num_trials": 50,
}


callbacks = [MyPrintingCallback(), checkpoint_callback, callback]

scheduler = ASHAScheduler(max_t=config["epochs"], grace_period=1, reduction_factor=2)

reporter = CLIReporter(
    parameter_columns=["lr", "batch_size"],
    metric_columns=["loss", "mean_accuracy", "training_iteration"],
)

# ray.init(local_mode=True, num_cpus=4, num_gpus=0)  # for debugging

# create versioning for multiple runs
def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    return [atoi(c) for c in re.split("(\d+)", text)]


log_dir = "../logs/StancePrediction_SemEval/lightning_logs/"
log_path = os.path.join(path, log_dir)
os.makedirs(os.path.dirname(log_path), exist_ok=True)
ver = os.listdir(os.path.join(path, log_dir))
ver.sort(key=natural_keys)
if ver:
    version = int(ver[-1].split("_", 2)[-1]) + 1
else:
    version = 0

In [ ]:
# start hyperparameter optimization
analysis = tune.run(
    tune.with_parameters(
        train_tune, callbacks=callbacks, epochs=config["epochs"], gpus=0
    ),
    config=config,
    num_samples=config["num_trials"],
    local_dir=os.path.join(path, "../logs/StancePrediction_SemEval/ray_results"),
    name="version_" + str(version),
    metric="loss",
    mode="min",
    scheduler=scheduler,
    progress_reporter=reporter,
)
# metric="loss", mode="min", scheduler=scheduler, progress_reporter=reporter

# get some information from the optimization
best_trial = analysis.best_trial  # Get best trial
best_config = analysis.best_config  # Get best trial's hyperparameters
best_logdir = analysis.best_logdir  # Get best trial's logdir
best_checkpoint = analysis.best_checkpoint  # Get best trial's best checkpoint
best_result = analysis.best_result  # Get best trial's last results
best_result_df = analysis.best_result_df  # Get best result as pandas dataframe

# Get a dataframe with the last results for each trial
df_results = analysis.results_df

# Get a dataframe of results for a specific score or mode
df = analysis.dataframe(metric="loss", mode="min")
# df2 = analysis.dataframe(metric="val_epoch_F1", mode="max") # check how to include multiple metrics


print("Best hyperparameters found were: ", analysis.best_config)

# save dataframe with results from hyperparameter search as csv?